# Cheatsheet for `pyspark`
A super-synthetic cheatsheet with the most important commands for `pyspark`

`.select('col1', 'col4', 'col7')` - to select only certain  <br>
`.distinct()` -  to select only distinct rows <br>
`.sort('col')` - to sort data in ascending order of `'col` <br>

## `pyspark.ml.feature`
A series of methods for transforming datasets.

**Label enconding** converts a categorical attribute to a number.
```
indexer = StringIndexer(inputCol=..., outputCol=...)
indexer = indexer.fit(df)
df_idx = indexer.transform(df)
```

**One-hot encoding**
```from pysparl.ml.feature import OneHotEncoderEstimator
ohe = OneHotEncoderEstimator(inputCols=[...], outputCols=[...])
ohe = ohe.fit(df)
df_ohe = ohe.transform(df)

**Bucketing**
```
bkt = Bucketizer(splits=[val1, val2, val3,...], inputCol='col', outputCol='col_bkt')
df = bkt.transform(df)
```

**Tokenizing & remove stop words**
```
tokenizer = Tokenizer(inputCol=..., outputCol=...)
tokenizer = tokenizer.fit(df)
df = tokenizer.transform(df)

remover = StopWordsRemover(inputCol=..., outputCol=...)
remover = remover.fit(df)
df = remover.transform(df)
```

**TF-IDF**
```
hasher = HashingTF(inputCol=..., outputCol=...)
hasher = hasher.fit(df)
df = hasher.transform(df)

idf = IDF(inputCol=..., outputCol=...)
idf = idf.fit(df)
df = idf.transform(df)
```

**Vector assembler** allows to assemble all the feature columns into a single `feature` column which is the required format by `pyspark`'s ML algorithm.

```
assemble = VectorAssembler(inputCols=[...], outputCol='features')
X_train = assemble.transform(df)
```

## `pyspark.ml.regression`
A series of ML techniques for regression

```
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

reg = LinearRegression(labelCol='col').fit(X_train)
pred = reg.transform(X_test)
RegressionEvaluator(labelCol='col').evaluate(pred)
```

- As a consequence of `.fit()`, `reg` contains an `.intercept` and an `.coefficients` attribute. Be carefule with the order of `.coefficients` elements, especially with one-hot encoded features.
- As a consequence of `.transform()`, `pred` contains a column named `prediction` which is the output of the model.
- `RegressionEvaluator` computes the RMSE by default, but could also compute `mae`, `r2`, `mse`.

## `pyspark.ml.classification`

```
from pyspark.ml.classification import GBTClassifier
form pyspark.ml.evaluation import BinaryClassificationEvaluator

gbt = GBTClassifier().fit(X_train)
preds = gbt.transform(X_test)
BinaryClassifierEvaluator().evaluate(preds)
```
- `BinaryClassifierEvaluator()` evaluates the `AUC` by default.

## `pyspark.ml.tuning`
A series of routines for cross-validation

```
reg = LinearRegression(labelCol='col')
evaluator = RegressionEvaluator(labelCol='col')

params = ParamGridBuilder()
params = params.addGrid(reg.fitIntercept, [True, False])
params = params.build()

cv = CrossValidator(estimator=reg, 
                    estimatorParamMaps=params, 
                    evaluator=evaluator, 
                    numFolds=5, 
                    seed=42)
cv.fit(X_train)
print(cv.bestModel)
print(cv.bestModel.stages) #if estimator is a pipeline
print(cv.bestModel.explainParam('fitIntercept))
print(cv.avgMetrics)
```